In [37]:
import pandas as pd
import pickle
import os
import random
import datetime as dt

In [52]:
repo_dir = "/Users/ameliachu/repos/nlu-reddit-toxicity-dataset"
labelled_data_dir = f"{repo_dir}/data/labelled/"
labelled_data_fnames = [f for f in os.listdir(labelled_data_dir)]
randomized_example_ids_path = f"{repo_dir}/data/randomized_example_ids.p"
interrater_assignment_path = f"{repo_dir}/data/interrater-reliability/interrater_assignment_2022-04-21.p"
print(labelled_data_fnames)
print(randomized_example_ids_path)

['yj2369_labelling_assignment_2022-04-13.csv', 'gm2858_labelling_assignment_2022-04-09.csv', 'yp2201_labelling_assignment_2022-04-09.csv', 'ac4119_labelling_assignment_2022-04-09.csv', 'ac4119_labelling_assignment_2022-04-13.csv', 'yp2201_labelling_assignment_2022-04-13.csv', 'gm2858_labelling_assignment_2022-04-13.csv', 'yj2369_labelling_assignment_2022-04-09.csv']
/Users/ameliachu/repos/nlu-reddit-toxicity-dataset/data/randomized_example_ids.p


In [53]:
example_indices = pickle.load( open(randomized_example_ids_path, "rb" ) )
interrater_assignment = pickle.load( open(interrater_assignment_path, "rb" ) )

In [4]:
selected_columns = ['example_id']

list_of_example_ids = [pd.read_csv(f"{labelled_data_dir}{fname}")[selected_columns] for fname in labelled_data_fnames]

In [11]:
list_of_example_ids.append(pd.read_csv(f"{repo_dir}/data/to_label/yp2201_labelling_assignment_2022-04-13")[selected_columns])

In [12]:
example_ids_pd = pd.concat(list_of_example_ids)

In [13]:
len(example_ids_pd)

1361

In [14]:
example_ids_pd.nunique()[0]

1358

In [15]:
new_ids = example_indices[1360:]

In [16]:
ids_that_should_have_labels = example_indices[:1360]

In [19]:
to_label_immediately = set(ids_that_should_have_labels) - set(example_ids_pd.example_id.values)

In [21]:
len(to_label_immediately)

597

In [30]:
# Grabbing the index of each example_id we are missing, so that we can replace the example_ids 
# in the correct order
indices_to_replace = [example_indices.index(i) for i in to_label_immediately]
ordered_indices = list(set(indices_to_replace))
missing_example_ids = [example_indices[i] for i in ordered_indices]

In [31]:
print(missing_example_ids[:5])

[375030, 200577, 329502, 511720, 52314]


In [34]:
rater_ids = ['ac4119', 'gm2858', 'yj2369','yp2201']
num_raters = len(rater_ids)

start_id = 0
batch_size = 140

supplmental_batches = []

for i in range(num_raters):
    end_id = start_id + batch_size
    supplmental_batches.append((start_id, end_id))
    start_id = end_id

In [35]:
print(supplmental_batches)

[(0, 140), (140, 280), (280, 420), (420, 560)]


In [38]:
random.shuffle(rater_ids)

In [39]:
assign_batches_to_raters =  list(zip(rater_ids,supplmental_batches))
print(assign_batches_to_raters)

[('ac4119', (0, 140)), ('yj2369', (140, 280)), ('yp2201', (280, 420)), ('gm2858', (420, 560))]


In [42]:
remaining_examples_to_backfill = missing_example_ids[560:]

In [43]:
len(remaining_examples_to_backfill)

37

In [47]:
remaining_backfill_path = f'{repo_dir}/data/backfill_example_ids.p'
# pickle.dump(remaining_examples_to_backfill, open(remaining_backfill_path, "wb" ) )

In [54]:
interrater_assignment

{'ac4119': [592265,
  121158,
  615201,
  48818,
  253577,
  210983,
  590568,
  76285,
  30108,
  349665,
  278618,
  93976,
  619248,
  127341,
  1138,
  316067,
  105684,
  480137,
  136994,
  155852,
  283669,
  119999,
  545491,
  312469,
  511369,
  79659,
  105683,
  70916,
  186797,
  307610,
  597550,
  305580,
  324017,
  196572,
  35731,
  81366,
  65971,
  229718,
  273130,
  309841,
  527590,
  148848,
  473471,
  461434,
  468289,
  73375,
  545157,
  527855,
  583609,
  363151,
  473334,
  198776,
  256825,
  488655,
  227207,
  259334,
  532332,
  104977,
  493832,
  592491,
  269910,
  616242,
  545067,
  71018,
  595070,
  347615,
  428916,
  527738,
  591714,
  556230,
  613683,
  511042,
  120753,
  245063,
  406404,
  21735,
  297420,
  229498,
  436980,
  158868,
  303224,
  8154,
  107332,
  56281,
  340510,
  142686,
  13089,
  589513,
  282097,
  129154,
  353614,
  491078,
  121387,
  261022,
  270201,
  515863,
  488812,
  61560,
  327873,
  87473,
  363879,


In [40]:
daily_discussion_data_fname = "daily_master_data_1614250838_1618692612.csv"
daily_discussion_data_path = f"{repo_dir}/data/{daily_discussion_data_fname}"
daily_discussion_data = pd.read_csv(daily_discussion_data_path).rename(columns={'Unnamed: 0':'example_id'})

In [55]:
to_label_dir = f"{repo_dir}/data/to_label"
current_date = dt.date.today()
labels = ['toxicity', 'severe_toxicity', 'identity_attack', 'insult', 'profanity', 'threat']

for rater_id, (start_ind, end_ind) in assign_batches_to_raters:
    fname = f"{rater_id}_labelling_assignment_{current_date}"
    export_location = f"{to_label_dir}/{fname}"
    assigned_indices = missing_example_ids[start_ind:end_ind]
    assigned_indices += interrater_assignment[rater_id]
    index_map = {}
    required_examples = []
    for ind in assigned_indices:
        ind_examples = [{
        'example_id': str(ind),
        'example_type': 'preceding',
        'body': daily_discussion_data[daily_discussion_data['example_id'] == ind-1]['body'].values[0]
    }, {
        'example_id': str(ind),
        'example_type': 'example',
        'body': daily_discussion_data[daily_discussion_data['example_id'] == ind]['body'].values[0]
    },
     {
        'example_id': str(ind),
        'example_type': 'following',
        'body': daily_discussion_data[daily_discussion_data['example_id'] == ind+1]['body'].values[0]
    }]
        required_examples += ind_examples 
    assigned_data = pd.DataFrame(required_examples)
    # Adding in the primary example info and the type of example for each row.
    assigned_data = assigned_data[['example_type','example_id','body']].reset_index()
    assigned_examples_pivot = assigned_data.pivot(index='example_id', columns='example_type', values='body').reset_index()[['example_id','preceding', 'example','following']]
    assigned_examples = assigned_examples_pivot.rename(columns={
         'preceding':'preceding_comment',
         'following':'following_comment',
         'example':'comment_for_evaluation'})
    for label in labels:
         assigned_examples[label] = ""
    assigned_examples.to_csv(export_location, index=False)
    

In [56]:
len(assigned_examples)

260